In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd

In [ ]:
namespace="broad-genomics-delivery"
workspace="Getz_IBM_CellLines_Exomes"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_WES_CN_HG38"
source="ibm"

intervals = '"gs://fc-secure-bb273170-1bd6-405c-82c7-67fb31fcda40/references/intervals/agilent_hg38_lifted_chrXX.no_header.list"'
readcountpon = '"gs://fc-secure-b82334d0-5e87-4e4d-8e0b-757b9a8aae6b/49f36c0e-5731-4228-b40b-882a182616c0/CNVSomaticPanelWorkflow/2875ee8d-c68f-45f9-b4c2-3683ea5dc42d/call-CreateReadCountPanelOfNormals/hg38_agilent_pon_XX.pon.hdf5"'

In [ ]:
wm = dm.WorkspaceManager(namespace, workspace)
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [ ]:
refsamples = refwm.get_samples()
refids = refsamples['participant'].tolist()
refids = [val[val.index('ACH'):] for val in refids]
refsamples

In [ ]:
samples = wm.get_samples()
samples = samples[samples['individual_alias'].str.contains('ACH')][~samples['individual_alias'].str.slice(0,10).isin(refids)]
samples['individual_alias']

In [ ]:
for ind, val in samples.iterrows():
    refsamples = refsamples.append(pd.Series({
        "CCLE_name": val['sample_alias'],
        "WES_bai": val['crai_or_bai_path'],
        "WES_bam": val['cram_or_bam_path'],
        "Source": source,
        "participant":val['individual_alias'][:10],
    },name=source+'_'+val['individual_alias'][:10]))

In [ ]:
refsamples.shape

In [ ]:
refwm.upload_samples(refsamples)

In [ ]:
a = [source+'_'+i for i in samples['individual_alias'].str.slice(0,10).tolist()]
a

In [ ]:
refwm.update_sample_set(sample_set_id="CCLE_19Q3interim",sample_ids=a)

In [ ]:
conf = refwm.get_config("CNV_sample_XX")
conf

In [ ]:
CNVSomaticPairWorkflow['inputs']['CNVSomaticPairWorkflow.tumor_bam_idx'] = "this.WES_bai"
CNVSomaticPairWorkflow['inputs']['CNVSomaticPairWorkflow.tumor_bam'] = "this.WES_bam"
CNVSomaticPairWorkflow['inputs']['CNVSomaticPairWorkflow.read_count_pon'] = readcountpon
CNVSomaticPairWorkflow['inputs']['CNVSomaticPairWorkflow.intervals'] = intervals
submission_id = refwm.create_submission(CNVSomaticPairWorkflow['name'],etype='sample_set',entity="CCLE_19Q3interim",expression='this.samples')
while wm.get_submission(submission_id)['status'] not in {'Done', 'Aborted'}:
    time.sleep(60)
assert wm.get_submission(submission_id)['status'] !='Aborted'